# KRX Options Expiration Calendar

In [ ]:
import pandas as pd
import json
import requests
import datetime
import re

## Market Holidays (2023 – 2027)
한국거래소 > KRX 시장 > 시장동향 > 증시일정 > [휴장일](https://open.krx.co.kr/contents/MKD/01/0110/01100305/MKD01100305.jsp)

In [ ]:
holidays_ser = pd.read_excel('KRX_market_holidays.xls').iloc[:, 0]
holidays = []

for date in holidays_ser:
    dt = datetime.datetime.strptime(date, '%Y-%m-%d').date()
    holidays.append(dt)

holidays

In [ ]:
# Create a trading days list of 2023 and onwards

start_date = datetime.date(2023, 1, 1)
end_date = holidays[-1]

trading_days_fwd = []
current_date = start_date

while current_date <= end_date:
    if current_date.weekday() < 5 and current_date not in holidays:
        trading_days_fwd.append(current_date)
    current_date += datetime.timedelta(days = 1)

In [ ]:
option_expiration_dates = []

for year in range(2023, 2028):
    for month in range(1, 13):
        for day in range(8, 15):
            dt = datetime.date(year, month, day)
            if (dt.weekday() == 3) & (dt not in holidays):
                option_expiration_dates.append(dt)
            elif (dt.weekday() == 3) & (dt in holidays):
                for days_offset in range(1, 4):
                    possible_date = dt - datetime.timedelta(days = days_offset)
                    if possible_date not in holidays:
                        option_expiration_dates.append(possible_date)
                        break

# Add 2025-10-02 as an exception (10/3 Fri. ~ 10/9 Thu. national holidays)
option_expiration_dates.append( datetime.date(2025, 10, 2) )
option_expiration_dates.sort()

option_expiration_dates

## Historical Trading Days (1999 – 2022)

In [ ]:
# c_amt_do is a DataFrame sourced directly using requests from KRX
trading_days = c_amt_do['일자']

days_list = []

for date in trading_days:
    dt = datetime.datetime.strptime(date, '%Y/%m/%d').date()
    days_list.append(dt)

In [ ]:
second_thursday = []

for year in range(1999, 2023):
    for month in range(1, 13):
        for day in range(8, 15):
            if datetime.date(year, month, day).weekday() == 3:
                second_thursday.append( datetime.date(year, month, day) )

second_thursday = second_thursday[4:]   # trading data available from 1999-04-26

In [ ]:
option_exp_date = []
for secthu in second_thursday:
    for days_offset in range(0, 4):
        possible_date = secthu - datetime.timedelta(days = days_offset)
        if possible_date in days_list:
            option_exp_date.append(possible_date)
            break
    else:
        raise ValueError(f"No valid option expiration date found for second Thursday {secthu}.")

## Options Expiration Dates

In [ ]:
option_exp_date = option_exp_date + option_expiration_dates
option_exp_date

---

## Time to Maturity

In [ ]:
trading_days = list(set(days_list + trading_days_fwd))
trading_days = sorted(trading_days)
trading_days = pd.Series(trading_days)

In [ ]:
days_until_exp = {}
index = 0

for i in trading_days:
    if i < option_exp_date[index]:
        days_until_exp[i] = trading_days.index[ trading_days == option_exp_date[index] ][0] - trading_days.index[ trading_days == i ][0]
    elif i == option_exp_date[index]:
        days_until_exp[i] = trading_days.index[ trading_days == option_exp_date[index] ][0] - trading_days.index[ trading_days == i ][0]
        index += 1

ser = pd.Series(days_until_exp)

In [ ]:
ser[-30:]